# HGT Experiments

In [1]:
import os
import sys

import numpy as np
import pandas as pd

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import torch
import torch_geometric as pyg
from torch_geometric.nn import Linear
from torch.nn import functional as F

from src import temporal_loader_v2 as tl
from src.utils import score
from hgt_conv import HGTConv

In [2]:
cuda_device = 2
pyg.seed_everything(42)

### Model

In [3]:
class HGT(torch.nn.Module):
    def __init__(self, data, hidden_channels, out_channels, num_heads, num_layers, num_features=-1):
        super().__init__()

        self.lin_dict = torch.nn.ModuleDict()
        for node_type in data.node_types:
            self.lin_dict[node_type] = Linear(num_features, hidden_channels)

        self.convs = torch.nn.ModuleList()
        for _ in range(num_layers):
            conv = HGTConv(hidden_channels, hidden_channels, data.metadata(),
                           num_heads, group='sum')
            self.convs.append(conv)

        self.lin = Linear(hidden_channels, out_channels)

    def forward(self, x_dict, edge_index_dict, target_nodetype='domain_node'):
        for node_type, x in x_dict.items():
            x_dict[node_type] = self.lin_dict[node_type](x).relu_()

        for conv in self.convs:
            x_dict = conv(x_dict, edge_index_dict)

        return self.lin(x_dict[target_nodetype])
    

def train(model, data, optimizer):
    model.train()
    optimizer.zero_grad()
    out = model(data.x_dict, data.edge_index_dict)
    mask = data['domain_node'].train_mask
    loss = F.cross_entropy(out[mask], data['domain_node'].y[mask])
    loss.backward()
    optimizer.step()
    return float(loss)


@torch.no_grad()
def test(model, data):
    model.eval()
    pred = model(data.x_dict, data.edge_index_dict).argmax(dim=-1)

    accs = []
    for split in ['train_mask', 'val_mask']:
        mask = data['domain_node'][split]
        acc = (pred[mask] == data['domain_node'].y[mask]).sum() / mask.sum()
        accs.append(float(acc))
    return accs

In [4]:
def experiment(start,end,test_list, model_type):
    kg_path = lambda graph_name: f'../../data/{graph_name}'

    dataset = tl.DNS(root=kg_path('DNS_2m'), start=start, end=end, test_list=test_list, balance_gt=False, domain_file='domains2.csv')
    data = dataset.train_data # training data
    
    model = HGT(data, hidden_channels=64, out_channels=2, num_heads=8, num_layers=2)

    if torch.cuda.is_available():
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        torch.cuda.set_device(cuda_device)

        data, model = data.to(device), model.to(device)

    with torch.no_grad():  # Initialize lazy modules.
        out = model(data.x_dict, data.edge_index_dict)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=0.001)

    for epoch in range(0, 201):
        loss = train(model, data, optimizer)
        train_acc, val_acc = test(model,data)
        if epoch % 20 == 0:
            print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train: {train_acc:.4f}, '
                f'Val: {val_acc:.4f}')
        
    model.eval()
    for index, test_data in enumerate(dataset.test_data):
        test_data = test_data.to(device)
        with torch.no_grad():
            pred = model(test_data.x_dict, test_data.edge_index_dict).argmax(dim=-1)
        mask = test_data['domain_node']['val_mask']
        scores = score(pred[mask],test_data['domain_node'].y[mask])
        with open("../../results_copy.csv", "a") as logger:
            logger.write("{},{},{},{},".format(model_type,start,end,index))
            logger.write(",".join(str(x) for x in scores.values()))
            logger.write('\n')


        for metric, val in scores.items():
            print(metric, ':{:.4f}'.format(val))
    
for model_type in ['hgt']:
    for i in range(5):
        experiment(i,i+6,[i+7,i+8], model_type)

Total labeled 897635
Labeled node count for 0, 6: 31778
Labeled node count for 0, 7: 2610
Labeled node count for 0, 8: 2083
Epoch: 000, Loss: 0.6929, Train: 0.5514, Val: 0.5380
Epoch: 020, Loss: 0.5718, Train: 0.6950, Val: 0.6875
Epoch: 040, Loss: 0.5288, Train: 0.7644, Val: 0.7580
Epoch: 060, Loss: 0.4724, Train: 0.7657, Val: 0.7574
Epoch: 080, Loss: 0.4428, Train: 0.7739, Val: 0.7659
Epoch: 100, Loss: 0.4316, Train: 0.7888, Val: 0.7816
Epoch: 120, Loss: 0.4233, Train: 0.7994, Val: 0.7923
Epoch: 140, Loss: 0.4153, Train: 0.8066, Val: 0.8030
Epoch: 160, Loss: 0.4072, Train: 0.8119, Val: 0.8080
Epoch: 180, Loss: 0.3998, Train: 0.8158, Val: 0.8131
Epoch: 200, Loss: 0.3937, Train: 0.8161, Val: 0.8159
tn, fp, fn, tp 1382 223 181 824
acc :0.8452
f1 :0.8458
auc :0.8405
prec :0.7870
recall :0.8199
fpr :0.1389
mi_f1 :0.8452
ma_f1 :0.8378
tn, fp, fn, tp 1128 181 157 617
acc :0.8377
f1 :0.8382
auc :0.8294
prec :0.7732
recall :0.7972
fpr :0.1383
mi_f1 :0.8377
ma_f1 :0.8273
Total labeled 897635
La

KeyboardInterrupt: 